<a href="https://colab.research.google.com/github/rocioromera911/Portafolio_RR/blob/main/test_opencv_dig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de Imágenes - Test Técnico OpenCV

In [ ]:
# ✅ Librerías necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure

In [ ]:
# ✅ Leer y redimensionar Test1.jpg
img = cv2.imread('/content/Test1.jpg')
img_resized = cv2.resize(img, (1024, 1024))
cv2.imwrite('/content/Test1_resized.jpg', img_resized)

In [ ]:
# ✅ Dividir en 4 cuadrantes
h, w = img_resized.shape[:2]
quads = [
    img_resized[0:h//2, 0:w//2],
    img_resized[0:h//2, w//2:],
    img_resized[h//2:, 0:w//2],
    img_resized[h//2:, w//2:]
]
plt.figure(figsize=(10, 10))
for i, quad in enumerate(quads):
    plt.subplot(2, 2, i+1)
    plt.imshow(cv2.cvtColor(quad, cv2.COLOR_BGR2RGB))
    plt.title(f'Cuadrante {i+1}')
    plt.axis('off')
plt.show()

In [ ]:
# ✅ Ecualización por canal
channels = cv2.split(img_resized)
colors = ['b', 'g', 'r']
plt.figure(figsize=(12, 4))
for i, (ch, color) in enumerate(zip(channels, colors)):
    eq = cv2.equalizeHist(ch)
    plt.subplot(1, 3, i+1)
    plt.hist(eq.ravel(), bins=256, color=color)
    plt.title(f'Canal {color.upper()} Ecualizado')
plt.tight_layout()
plt.show()

In [ ]:
# ✅ Procesar MASK.tif
mask = cv2.imread('/content/MASK.tif', cv2.IMREAD_GRAYSCALE)
binary = (mask * 255).astype(np.uint8)  # convertir de [0,1] a [0,255]
labels = measure.label(binary, connectivity=2)
props = measure.regionprops(labels)
if len(props) > 0:
    total_objects = len(props)
    avg_area = np.mean([p.area for p in props])
    print(f'Total de objetos: {total_objects}')
    print(f'Área promedio: {avg_area:.2f}')
else:
    print('⚠️ No se encontraron objetos en la máscara.')

In [ ]:
# ✅ Apertura morfológica
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
opened = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
plt.imshow(opened, cmap='gray')
plt.title('Máscara con apertura')
plt.axis('off')
plt.show()

In [ ]:
# ✅ Fusionar máscara azul sobre Test1
mask_color = np.zeros_like(img_resized)
mask_color[:, :, 0] = binary
fused = cv2.addWeighted(img_resized, 1.0, mask_color, 0.5, 0)
plt.imshow(cv2.cvtColor(fused, cv2.COLOR_BGR2RGB))
plt.title('Fusión con máscara azul')
plt.axis('off')
plt.show()

## Descripción técnica de la máscara:
- Tipo de variable: Binaria (originalmente con valores 0 y 1, convertida a 0 y 255)
- Conteo de objetos realizado con `skimage.measure.label()`
- Promedio de área obtenido con `.area` de `regionprops()`
- Apertura morfológica aplicada con OpenCV `cv2.morphologyEx()`
- Se utilizó un kernel elíptico de 5x5 para la operación morfológica.